In [1]:
from model import QTModel
import torch
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from nltk.tokenize import sent_tokenize as ST

In [2]:
W2V_PATH = "/home/jingjing/Desktop/InferSent-master/dataset/GloVe/glove.840B.300d.txt"

In [3]:
#f = open('/home/jingjing/Desktop/small.txt', 'r')
with open('/home/jingjing/Desktop/S2V-master/BookCorpus/try.txt', 'r') as in_file:
    text = in_file.read()
    sents = ST(text)

In [4]:
#paragraphs = splat
data = sents
len(data)

18

In [5]:
sentences = ['Doctors lead a hard life.',
             'Their life is very busy.',
             'They get up early in the morning and go to the hospital.']

In [6]:
f = QTModel()
f.set_w2v_path(W2V_PATH)

In [7]:
def make_target(context_size, dim):
    targets = np.zeros((dim, dim))
    ctxt_sent_pos = list(range(-context_size, context_size+1))
    ctxt_sent_pos.remove(0)
    for ctxt in ctxt_sent_pos:
        targets += np.eye(dim, k=ctxt)
    targets_sum = np.sum(targets,axis=1, keepdims=True)
    targets = targets / targets_sum
    targets = torch.from_numpy(targets)
    return targets

In [8]:
def loss_fn(pred, target):
    mask = 1 - torch.diag(torch.ones(pred.size(1)))
    npred = pred * mask
    s_pred = F.softmax(npred, -1)
    ln = nn.BCEWithLogitsLoss()
    #ln = nn.KLDivLoss()
    losses = ln(s_pred, target)
    return losses

In [9]:
def nln(pred, target):
    mask = 1 - torch.diag(torch.ones(pred.size(1)))
    npred = pred * mask
    s_pred = F.softmax(npred, -1)
    #print(s_pred)
    #print(torch.log(s_pred))
    product = targets.float() * torch.log(s_pred)
    #print(targets)
    #print(product)
    loss = torch.mean(-torch.sum(product,1))
    return loss

In [11]:
with torch.no_grad():
    f.build_vocab(sentences, True)
    embs = f(sentences, 400, True, True)
    #print(embs)
    targets = make_target(1, len(sentences))
    loss = nln(embs, targets.float())
    
    print("loss before training: ", loss)

Found 23(/23) words with w2v vectors
Vocab size : 23
Found 23(/23) words with w2v vectors
Vocab size : 23
Nb words kept : 31/31 (100.0%)
Speed : 82.9 sentences/s (cpu mode, bsize=400)
Nb words kept : 31/31 (100.0%)
Speed : 88.3 sentences/s (cpu mode, bsize=400)
loss before training:  tensor(1.1495)


In [12]:
optimizer = optim.Adam(f.parameters(), lr=0.0005)
nn.utils.clip_grad_norm_(f.parameters(), 5)

0.0

In [13]:
print(len(data))

18


In [14]:
for epoch in range(10):
    optimizer.zero_grad()
    if epoch == 0:
        f.build_vocab(data, True)
    targets = make_target(1, len(data))
    scores = f(data, 400)
    loss = nln(scores, targets.float())
    print(loss)
    loss.backward()
    optimizer.step()

Found 186(/188) words with w2v vectors
Vocab size : 186
Found 186(/188) words with w2v vectors
Vocab size : 186
tensor(2.9004)
tensor(2.8098)
tensor(2.7037)
tensor(2.4710)
tensor(2.2661)
tensor(1.9940)
tensor(1.6606)
tensor(1.4069)
tensor(1.3383)
tensor(1.0879)


In [15]:
with torch.no_grad():
    #f.build_vocab(sentences, True)
    embeddings = f(sentences, 400)
    targets = make_target(1, len(sentences))
    loss = loss_fn(embeddings, targets.float())
    print("loss after training: ", loss)

loss after training:  tensor(0.7034)


In [16]:
saved = "/home/jingjing/Desktop/emb/saved/qt.pkl"
torch.save(f.state_dict(), saved)

In [ ]:
embs1 = f.f.encode(sentences, 400)

In [ ]:
emb2 = f.g.encode(sentences, 400)

In [ ]:
embs1

In [ ]:
emb2

In [ ]:
torch.cat((embs1, emb2),1)